In [1]:
import nltk
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import re

In [2]:
import statistics

In [3]:
latlib_df = pd.read_pickle("/Users/francesharrington/Documents/ling1340/Latin-Vocabulary-Analysis/data/latinlibrary_data.pkl")

In [4]:
ne_df = pd.read_pickle("/Users/francesharrington/Documents/ling1340/Latin-Vocabulary-Analysis/data/latinlibrary_ne_data.pkl")

In [5]:
#here's some of the data:
latlib_df.head()

,title,text,text_raw,tokens,token_count
0,Ammianus: Liber XXXI,AMMIANI MARCELLINI HISTORIAE LIBER XXXI ...,"[AMMIANI MARCELLINI HISTORIAE LIBER XXXI\n, \n...","[AMMIANI, MARCELLINI, HISTORIAE, LIBER, XXXI, ...",11030
1,Valerius Maximus I,VALERI MAXIMI FACTORVM ET DICTORVM MEMORABILIV...,[VALERI MAXIMI FACTORVM ET DICTORVM MEMORABILI...,"[VALERI, MAXIMI, FACTORVM, ET, DICTORVM, MEMOR...",10524
2,Valerius Maximus VIII,VALERI MAXIMI FACTORVM ET DICTORVM MEMORABILIV...,[VALERI MAXIMI FACTORVM ET DICTORVM MEMORABILI...,"[VALERI, MAXIMI, FACTORVM, ET, DICTORVM, MEMOR...",10337
3,Valerius Maximus IV,VALERI MAXIMI FACTORVM ET DICTORVM MEMORABILIV...,[VALERI MAXIMI FACTORVM ET DICTORVM MEMORABILI...,"[VALERI, MAXIMI, FACTORVM, ET, DICTORVM, MEMOR...",10852
4,Valerius Maximus VII,VALERI MAXIMI FACTORVM ET DICTORVM MEMORABILIV...,[VALERI MAXIMI FACTORVM ET DICTORVM MEMORABILI...,"[VALERI, MAXIMI, FACTORVM, ET, DICTORVM, MEMOR...",8918


In [6]:
ne_df.head()

,name,era
0,AMMIANVS MARCELLINVS,(4th century A.D.)
1,VALERIVS MAXIMVS,(1st century A.D.)
2,GAIVS VALERIVS FLACCVS,(died c. 90 A.D.)
4,P. VERGILIVS MARO,(70 – 19 B.C.)
6,ALBIVS TIBVLLVS,(c. 54 – 19 B.C.)


Here's some basic info on the data contained in latlib_df:

In [7]:
#overall shape of the dataframe:
latlib_df.shape

(672, 5)

In [8]:
latlib_df.describe()

,token_count
count,672.000000
mean,6331.989583
std,5564.159005
min,0.000000
25%,1681.500000
50%,5578.500000
75%,8590.500000
max,38414.000000


In [9]:
latlib_df.info()
#good! no null objects

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 672 entries, 0 to 671
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        672 non-null    object
 1   text         672 non-null    object
 2   text_raw     672 non-null    object
 3   tokens       672 non-null    object
 4   token_count  672 non-null    int64 
dtypes: int64(1), object(4)
memory usage: 26.4+ KB


let's work on seperating works by style (prose or poetry):

In [10]:
#use Aeneid (P. VERGILI MARONIS AENEIDOS) + Ovid (P. OVIDI NASONIS) for verse examples

In [11]:
metamorphoses_ovid = latlib_df[latlib_df['text'].str.contains('P. OVIDI NASONIS METAMORPHOSEON')]

In [5]:
aeneid = latlib_df[latlib_df['text'].str.contains('P. VERGILI MARONIS AENEIDOS')]

In [13]:
aeneid

,title,text,text_raw,tokens,token_count
123,P. VERGILI MARONIS AENEIDOS LIBER DVODECIMVS,P. VERGILI MARONIS AENEIDOS LIBER DVODECIMVS ...,[P. VERGILI MARONIS AENEIDOS LIBER DVODECIMVS\...,"[P., VERGILI, MARONIS, AENEIDOS, LIBER, DVODEC...",7262
124,P. VERGILI MARONIS AENEIDOS LIBER VNDECIMVS,P. VERGILI MARONIS AENEIDOS LIBER VNDECIMVS O...,[P. VERGILI MARONIS AENEIDOS LIBER VNDECIMVS\n...,"[P., VERGILI, MARONIS, AENEIDOS, LIBER, VNDECI...",6884
125,P. VERGILI MARONIS AENEIDOS LIBER OCTAVVS,P. VERGILI MARONIS AENEIDOS LIBER OCTAVVS Ut ...,[P. VERGILI MARONIS AENEIDOS LIBER OCTAVVS\n\n...,"[P., VERGILI, MARONIS, AENEIDOS, LIBER, OCTAVV...",5420
126,Aeneid IV,P. VERGILI MARONIS AENEIDOS LIBER QVARTVS At ...,"[P. VERGILI MARONIS AENEIDOS LIBER QVARTVS\n, ...","[P., VERGILI, MARONIS, AENEIDOS, LIBER, QVARTV...",5408
127,P. VERGILI MARONIS AENEIDOS LIBER DECIMVS,P. VERGILI MARONIS AENEIDOS LIBER DECIMVS Pan...,"[P. VERGILI MARONIS AENEIDOS LIBER DECIMVS\n, ...","[P., VERGILI, MARONIS, AENEIDOS, LIBER, DECIMV...",6985
128,P. VERGILI MARONIS AENEIDOS LIBER SEPTIMVS,P. VERGILI MARONIS AENEIDOS LIBER SEPTIMVS Tu...,"[P. VERGILI MARONIS AENEIDOS LIBER SEPTIMVS\n,...","[P., VERGILI, MARONIS, AENEIDOS, LIBER, SEPTIM...",6058
129,P. VERGILI MARONIS AENEIDOS LIBER NONVS,P. VERGILI MARONIS AENEIDOS LIBER NONVS Atque...,"[P. VERGILI MARONIS AENEIDOS LIBER NONVS\n, \n...","[P., VERGILI, MARONIS, AENEIDOS, LIBER, NONVS,...",6222
130,P. VERGILI MARONIS AENEIDOS LIBER TERTIVS,P. VERGILI MARONIS AENEIDOS LIBER TERTIVS Pos...,"[P. VERGILI MARONIS AENEIDOS LIBER TERTIVS\n, ...","[P., VERGILI, MARONIS, AENEIDOS, LIBER, TERTIV...",5318
131,P. VERGILI MARONIS AENEIDOS LIBER SECVNDVS,P. VERGILI MARONIS AENEIDOS LIBER SECVNDVS Co...,"[P. VERGILI MARONIS AENEIDOS LIBER SECVNDVS\n,...","[P., VERGILI, MARONIS, AENEIDOS, LIBER, SECVND...",6069
132,P. VERGILI MARONIS AENEIDOS LIBER SEXTVS,P. VERGILI MARONIS AENEIDOS LIBER SEXTVS Sic ...,"[P. VERGILI MARONIS AENEIDOS LIBER SEXTVS\n, \...","[P., VERGILI, MARONIS, AENEIDOS, LIBER, SEXTVS...",6944


In [7]:
#[x for x in aeneid.text_raw]
#I want raw text for this because there are line divisions--helpful for distinguishing verse from prose

In [15]:
[len(x) for x in aeneid.text_raw]
#this gets me how many lines per-work there are (good metric for differentiating epic?)

[964, 924, 741, 715, 920, 833, 825, 727, 811, 905, 767, 880]

In [16]:
len([line for line in aeneid.text_raw][0][3])
#basically I want iterate like this for each line and find the average length of each line overall

44

In [17]:
count = 0
for i in aeneid.text_raw:
    count += len(i)
print(count) #this is how many lines are in the aeneid in total

10012


In [18]:
def MaxLineLength(l):
    return(len(max(l, key=len)))

In [19]:
max(aeneid.text_raw.map(MaxLineLength))
#so the longest line is 87 characters long, that's my threshold for a verse line (depending on Ovid I guess)

87

In [20]:
max(metamorphoses_ovid.text_raw.map(MaxLineLength)) #yep, 87 it is!

81

now to actually use that number to quantify prose v. verse!

In [21]:
latlib_df['max_line_length'] = latlib_df.text_raw.map(MaxLineLength)

In [22]:
latlib_df.head(5)

,title,text,text_raw,tokens,token_count,max_line_length
0,Ammianus: Liber XXXI,AMMIANI MARCELLINI HISTORIAE LIBER XXXI ...,"[AMMIANI MARCELLINI HISTORIAE LIBER XXXI\n, \n...","[AMMIANI, MARCELLINI, HISTORIAE, LIBER, XXXI, ...",11030,774
1,Valerius Maximus I,VALERI MAXIMI FACTORVM ET DICTORVM MEMORABILIV...,[VALERI MAXIMI FACTORVM ET DICTORVM MEMORABILI...,"[VALERI, MAXIMI, FACTORVM, ET, DICTORVM, MEMOR...",10524,2080
2,Valerius Maximus VIII,VALERI MAXIMI FACTORVM ET DICTORVM MEMORABILIV...,[VALERI MAXIMI FACTORVM ET DICTORVM MEMORABILI...,"[VALERI, MAXIMI, FACTORVM, ET, DICTORVM, MEMOR...",10337,1308
3,Valerius Maximus IV,VALERI MAXIMI FACTORVM ET DICTORVM MEMORABILIV...,[VALERI MAXIMI FACTORVM ET DICTORVM MEMORABILI...,"[VALERI, MAXIMI, FACTORVM, ET, DICTORVM, MEMOR...",10852,2034
4,Valerius Maximus VII,VALERI MAXIMI FACTORVM ET DICTORVM MEMORABILIV...,[VALERI MAXIMI FACTORVM ET DICTORVM MEMORABILI...,"[VALERI, MAXIMI, FACTORVM, ET, DICTORVM, MEMOR...",8918,2141


In [23]:
latlib_df.drop(latlib_df.index[latlib_df['token_count']<=10], inplace=True)


In [24]:
def Poems(l):
    if l <= 87:
        return('verse')
    else:
        return('prose')


In [25]:
latlib_df['style'] = latlib_df.max_line_length.map(Poems)

## below I'm going to add the aspects of ne_df to latlib_df for easier analysis:

In [26]:
ne_df['name'] = [w[:-2] for w in ne_df.name]

In [27]:
def MatchName(l):
    for name in ne_df.name:
        last_word = name.split()[len(name.split()) - 1]
        if last_word in l:
            return name
            return "None"

In [28]:
latlib_df['author'] = latlib_df.text.map(MatchName)

to put the eras in the main df it's not going to be elegant, but I want to move on with my analysis so I'm going to merge the info into a new column for an easier method:

In [29]:
ne_df['merged'] = ne_df['era']+ne_df['name']

In [30]:
def MatchEra(l):
    for x in ne_df.merged:
        last_word = x.split()[len(x.split()) - 1]
        if last_word in l:
            clean = re.compile(r'\)\w* \w* \w*|\)\w* \w*|\)\w+\. \w* \w*|\)\w+|\(')
            return(clean.sub(' ', x))
        

In [31]:
latlib_df['era'] = latlib_df.text.map(MatchEra)

In [32]:
latlib_df = latlib_df.fillna('none') #just in case, I think there's some Suetonius that's wonky

That wasn't very pretty, but it did the trick and now the df itself looks nice!

In [35]:
latlib_df.sample(5)

,title,text,text_raw,tokens,token_count,max_line_length,style,author,era
610,Cicero: Topica,M. TVLLI CICERONIS TOPICA [1] Maiores nos res...,"[M. TVLLI CICERONIS TOPICA\n, \n [1] Maiores n...","[M., TVLLI, CICERONIS, TOPICA, [, 1, ], Maiore...",8397,2068,prose,M. TVLLIVS CICE,106 – 43 B.C.
125,P. VERGILI MARONIS AENEIDOS LIBER OCTAVVS,P. VERGILI MARONIS AENEIDOS LIBER OCTAVVS Ut ...,[P. VERGILI MARONIS AENEIDOS LIBER OCTAVVS\n\n...,"[P., VERGILI, MARONIS, AENEIDOS, LIBER, OCTAVV...",5420,67,verse,P. VERGILIVS MA,70 – 19 B.C.
88,Terence: Hecyra,P. TERENTI AFRI HECYRA ...,"[P. TERENTI AFRI HECYRA\n, \n, \n, \n, \n, \n,...","[P., TERENTI, AFRI, HECYRA, Vxorem, ducit, Pam...",9312,78,verse,P. TERENTIVS AF,195/185 – c. 159 B.C.
620,Cicero: In Pisonem,M. TVLLI CICERONIS IN L. CALPVRNIVM PISONEM OR...,[M. TVLLI CICERONIS IN L. CALPVRNIVM PISONEM O...,"[M., TVLLI, CICERONIS, IN, L., CALPVRNIVM, PIS...",13184,2415,prose,TITVS LVCRETIVS CAR,c. 94 – c. 49 B.C.
622,Cicero: De Provinciis Consularibus,M. TVLLI CICERONIS DE PROVINCIIS CONSVLARIBUS ...,[M. TVLLI CICERONIS DE PROVINCIIS CONSVLARIBUS...,"[M., TVLLI, CICERONIS, DE, PROVINCIIS, CONSVLA...",6244,1163,prose,M. TVLLIVS CICE,106 – 43 B.C.


## sword word analysis:

In [36]:
latlib_df['term'] = ''

In [37]:
#gladius:
gladius = latlib_df[(latlib_df.text.str.contains(pat = 'gladius', case=False)) | (latlib_df.text.str.contains(pat = 'gladii', case=False)) | (latlib_df.text.str.contains(pat = 'gladi', case=False)) | (latlib_df.text.str.contains(pat = 'gladio', case=False)) | (latlib_df.text.str.contains(pat = 'gladium', case=False)) | (latlib_df.text.str.contains(pat = 'gladie', case=False)) | (latlib_df.text.str.contains(pat = 'gladiorum', case=False)) | (latlib_df.text.str.contains(pat = 'gladiis', case=False)) | (latlib_df.text.str.contains(pat = 'gladios', case=False))]

In [38]:
#ferrum:
ferrum = latlib_df[(latlib_df.text.str.contains(pat = 'ferrum', case=False)) | (latlib_df.text.str.contains(pat = 'ferri', case=False)) | (latlib_df.text.str.contains(pat = 'ferro', case=False)) | (latlib_df.text.str.contains(pat = 'ferra', case=False)) | (latlib_df.text.str.contains(pat = 'ferrorum', case=False)) | (latlib_df.text.str.contains(pat = 'ferris', case=False))]

In [39]:
#mucro:
mucro = latlib_df[(latlib_df.text.str.contains(pat = 'mucro', case=False)) | (latlib_df.text.str.contains(pat = 'mucronis', case=False)) | (latlib_df.text.str.contains(pat = 'mucroni', case=False)) | (latlib_df.text.str.contains(pat = 'mucronem', case=False)) | (latlib_df.text.str.contains(pat = 'mucrone', case=False)) | (latlib_df.text.str.contains(pat = 'mucrones', case=False)) | (latlib_df.text.str.contains(pat = 'mucronum', case=False)) | (latlib_df.text.str.contains(pat = 'mucronibus', case=False))]

In [40]:
#ensis:
ensis = latlib_df[(latlib_df.text.str.contains(pat = 'ensis', case=False)) | (latlib_df.text.str.contains(pat = 'ensi', case=False)) | (latlib_df.text.str.contains(pat = 'ensem', case=False)) | (latlib_df.text.str.contains(pat = 'ense', case=False)) | (latlib_df.text.str.contains(pat = 'enses', case=False)) | (latlib_df.text.str.contains(pat = 'ensium', case=False)) | (latlib_df.text.str.contains(pat = 'ensibus', case=False))]

In [41]:
#capulus:
capulus = latlib_df[(latlib_df.text.str.contains(pat = 'capulus', case=False)) | (latlib_df.text.str.contains(pat = 'capuli', case=False)) | (latlib_df.text.str.contains(pat = 'capulo', case=False)) | (latlib_df.text.str.contains(pat = 'capulum', case=False)) | (latlib_df.text.str.contains(pat = 'capulorum', case=False)) | (latlib_df.text.str.contains(pat = 'capulis', case=False)) | (latlib_df.text.str.contains(pat = 'capulos', case=False))]

In [42]:
#let's see how many occurences of each word there are:
print('gladius: ',len(gladius))
print('ferrum: ',len(ferrum))
print('mucro: ',len(mucro))
print('ensis: ',len(ensis))
print('capulus: ',len(capulus)) #'capulus' is the least common, only appearing 48 times, let's look at tht data!

gladius:  277
ferrum:  462
mucro:  89
ensis:  573
capulus:  48


In [43]:
capulus.head()

,title,text,text_raw,tokens,token_count,max_line_length,style,author,era,term
1,Valerius Maximus I,VALERI MAXIMI FACTORVM ET DICTORVM MEMORABILIV...,[VALERI MAXIMI FACTORVM ET DICTORVM MEMORABILI...,"[VALERI, MAXIMI, FACTORVM, ET, DICTORVM, MEMOR...",10524,2080,prose,VALERIVS MAXIM,1st century A.D.,
25,Statius: Achilleid II,PVBLIVS PAPINIVS STATIVS ACHILLEID II Exuit im...,"[PVBLIVS PAPINIVS STATIVS, \nACHILLEID II\n, \...","[PVBLIVS, PAPINIVS, STATIVS, ACHILLEID, II, Ex...",1243,71,verse,PVBLIVS PAPINIVS STATI,c. 45 – 96 A.D.,
31,Statius: Thebaid XI,PVBLIVS PAPINIVS STATIVS THEBAID XI Postquam m...,"[PVBLIVS PAPINIVS STATIVS, \nTHEBAID XI\n, \nP...","[PVBLIVS, PAPINIVS, STATIVS, THEBAID, XI, Post...",6017,72,verse,PVBLIVS PAPINIVS STATI,c. 45 – 96 A.D.,
34,Statius: Thebaid XII,PVBLIVS PAPINIVS STATIVS THEBAID XII Nondum cu...,"[PVBLIVS PAPINIVS STATIVS, \nTHEBAID XII\n, \n...","[PVBLIVS, PAPINIVS, STATIVS, THEBAID, XII, Non...",6244,72,verse,PVBLIVS PAPINIVS STATI,c. 45 – 96 A.D.,
35,Statius: Thebaid IX,PVBLIVS PAPINIVS STATIVS THEBAID IX Asperat Ao...,"[PVBLIVS PAPINIVS STATIVS, \nTHEBAID IX\n, \nA...","[PVBLIVS, PAPINIVS, STATIVS, THEBAID, IX, Aspe...",6925,74,verse,PVBLIVS PAPINIVS STATI,c. 45 – 96 A.D.,


Just from looking at these titles there's a pretty fair distribution of genre between the works (some philosophy, epic poetry, history, and even comedy plays!)

In [44]:
print(len(gladius[gladius['style']=='prose']))
print(len(gladius[gladius['style']=='verse']))
#the results here are skewed highly in favor of prose

221
56


In [45]:
print(len(ferrum[ferrum['style']=='prose']))
print(len(ferrum[ferrum['style']=='verse']))
#here there's more in the prose category

309
153


In [46]:
print(len(mucro[mucro['style']=='prose']))
print(len(mucro[mucro['style']=='verse']))
#it's closer here, but still in favor of prose

58
31


In [47]:
print(len(ensis[ensis['style']=='prose']))
print(len(ensis[ensis['style']=='verse']))
#heavily in favor of prose

406
167


In [48]:
print(len(capulus[capulus['style']=='prose']))
print(len(capulus[capulus['style']=='verse']))
#interestingly enough, verse is more popular than prose here

19
29


because the results are overwhelmingly in favor of prose in most categories let's look at how they match up in latlib_df

In [49]:
print(len(latlib_df[latlib_df['style']=='prose']))
print(len(latlib_df[latlib_df['style']=='verse']))
#so prose definitely shows more than verse, which is not super surprising

425
185


In [50]:
p_n = len(latlib_df[latlib_df['style']=='prose'])/len(latlib_df)*100
v_n = len(latlib_df[latlib_df['style']=='verse'])/len(latlib_df)*100
print('percentage of prose works in data: ',p_n,'%')
print('percentage of verse works in data: ',v_n,'%')

percentage of prose works in data:  69.67213114754098 %
percentage of verse works in data:  30.327868852459016 %


In [51]:
gladius.term = ['gladius' for x in gladius.term]
ferrum.term = ['ferrum' for x in ferrum.term]
mucro.term = ['mucro' for x in mucro.term]
ensis.term = ['ensis' for x in ensis.term]
capulus.term = ['capulus' for x in capulus.term]

/Users/francesharrington/opt/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:5168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [52]:
ensis.head(5)

,title,text,text_raw,tokens,token_count,max_line_length,style,author,era,term
0,Ammianus: Liber XXXI,AMMIANI MARCELLINI HISTORIAE LIBER XXXI ...,"[AMMIANI MARCELLINI HISTORIAE LIBER XXXI\n, \n...","[AMMIANI, MARCELLINI, HISTORIAE, LIBER, XXXI, ...",11030,774,prose,AMMIANVS MARCELLIN,4th century A.D.,ensis
1,Valerius Maximus I,VALERI MAXIMI FACTORVM ET DICTORVM MEMORABILIV...,[VALERI MAXIMI FACTORVM ET DICTORVM MEMORABILI...,"[VALERI, MAXIMI, FACTORVM, ET, DICTORVM, MEMOR...",10524,2080,prose,VALERIVS MAXIM,1st century A.D.,ensis
2,Valerius Maximus VIII,VALERI MAXIMI FACTORVM ET DICTORVM MEMORABILIV...,[VALERI MAXIMI FACTORVM ET DICTORVM MEMORABILI...,"[VALERI, MAXIMI, FACTORVM, ET, DICTORVM, MEMOR...",10337,1308,prose,VALERIVS MAXIM,1st century A.D.,ensis
3,Valerius Maximus IV,VALERI MAXIMI FACTORVM ET DICTORVM MEMORABILIV...,[VALERI MAXIMI FACTORVM ET DICTORVM MEMORABILI...,"[VALERI, MAXIMI, FACTORVM, ET, DICTORVM, MEMOR...",10852,2034,prose,VALERIVS MAXIM,1st century A.D.,ensis
4,Valerius Maximus VII,VALERI MAXIMI FACTORVM ET DICTORVM MEMORABILIV...,[VALERI MAXIMI FACTORVM ET DICTORVM MEMORABILI...,"[VALERI, MAXIMI, FACTORVM, ET, DICTORVM, MEMOR...",8918,2141,prose,VALERIVS MAXIM,1st century A.D.,ensis


In [53]:
df_swords = pd.concat([gladius, ferrum, mucro, ensis, capulus])

In [54]:
df_swords.sample(5)

,title,text,text_raw,tokens,token_count,max_line_length,style,author,era,term
418,Pliny the Younger,C. PLINII CAECILII SECVNDI EPISTVLARVM LIBER S...,[C. PLINII CAECILII SECVNDI EPISTVLARVM LIBER ...,"[C., PLINII, CAECILII, SECVNDI, EPISTVLARVM, L...",8125,502,prose,VALERIVS MAXIM,1st century A.D.,ensis
401,Plautus: Casina,T. MACCI PLAVTI CASINA OLYMPIO VILICVS CHALI...,"[T. MACCI PLAVTI CASINA\n, \n, \n, \nOLYMPIO V...","[T., MACCI, PLAVTI, CASINA, OLYMPIO, VILICVS, ...",9591,84,verse,P. VERGILIVS MA,70 – 19 B.C.,gladius
200,Justin XXVII,HISTORIARUM PHILIPPICARUM IN EPITOMEN REDACTI ...,[HISTORIARUM PHILIPPICARUM IN EPITOMEN REDACTI...,"[HISTORIARUM, PHILIPPICARUM, IN, EPITOMEN, RED...",784,237,prose,TITVS LVCRETIVS CAR,c. 94 – c. 49 B.C.,ensis
277,Nepos: Life of Miltiades,"CORNELI NEPOTIS MILTIADES [1] MILTIADES, Cimo...","[CORNELI NEPOTIS MILTIADES\n\n, \n[1] MILTIAD...","[CORNELI, NEPOTIS, MILTIADES, [, 1, ], MILTIAD...",1636,395,prose,CORNELIVS NEP,c. 100 – 24 B.C.,ferrum
467,"Silius, Liber XIV",SILIUS ITALICUS PUNICA LIBER DECIMUS QUARTUS F...,"[SILIUS ITALICUS, PUNICA LIBER DECIMUS QUARTUS...","[SILIUS, ITALICUS, PUNICA, LIBER, DECIMUS, QUA...",4952,66,verse,SILIVS ITALIC,25/6 – 101 A.D.,gladius


In [56]:
gladius[gladius['era']]

KeyError: "None of [Index([' 4th century A.D. ', ' 1st century A.D. ', ' 1st century A.D. ',\n       ' 1st century A.D. ', ' 1st century A.D. ', ' 1st century A.D. ',\n       ' died c. 90 A.D. ', ' died c. 90 A.D. ', ' c. 80/70 – c. 25 B.C. ',\n       ' 1st century A.D. ',\n       ...\n       'none', 'none', ' 70 – 19 B.C. ', 'none', ' 70 – 19 B.C. ',\n       ' 70 – 19 B.C. ', ' c. A.D. 125 – after 180 ',\n       ' c. A.D. 125 – after 180 ', ' c. A.D. 125 – after 180 ',\n       ' c. A.D. 125 – after 180 '],\n      dtype='object', length=277)] are in the [columns]"

In [ ]:
#updated data samples so they reflect my current data:
examples = latlib_df.head(10)
examples.to_pickle("/Users/francesharrington/Documents/ling1340/Latin-Vocabulary-Analysis/data_samples/examples.pkl")